# Yosys inference of memories

This example demonstrates direct inference of a dual port (simplex) memory description into a FPGA hard memory block (here: Lattice ECP5 DP16KD primitive).
It requires a recent (>= v0.12) release of yosys. Proper TDP inference may only be covered with a patched version.

**Not applicable to binder demo**

In [17]:
from cyhdl import *
from myirl.test.test_array import r1w1, SigArray

## Simple read and write port memory

This implementation uses bypass logic when a write is occuring during a read from the same address.

In [18]:
@block
def r1w1(
        clk : ClkSignal,
        we  : Signal,
        addr_r: Signal,
        addr_w: Signal,
        din: Signal,
        dout: Signal.Output,
        ram : SigArray,
        MODE = False, # Conditional compilation flag w/o type annotation
        DWIDTH=16, AWIDTH=10,
        TRANSPARENCY = False
    ):

    print("CALLED MEMORY INSTANCE", DWIDTH)
    
    if TRANSPARENCY:
        @always(clk.posedge)
        def mem_rw_transparent():
            if we and addr_w == addr_r:
                dout.next = din  #Forward
            else:
                dout.next = ram[addr_r][DWIDTH:]

            if we:
                ram[addr_w].next = din
    else:
        @always(clk.posedge)
        def mem_rw():
            dout.next = ram[addr_r][DWIDTH:]
            
            if we:
                ram[addr_w].next = din        
        
    return instances()


In [19]:
def memtest(MODE = 0, STYLE = 1, data_w=16, addr_w=6, mem = r1w1, TRANSPARENT = True):
    c = ClkSignal(name = 'clk')
    c.init = True
    wren = Signal(bool(), name = 'we')
    ra, wa = [ Signal(intbv()[addr_w:]) for n in ['addr_write', 'addr_read'] ]
    a, q = [ Signal(intbv()[data_w:]) for n in ['a', 'q'] ]

    ram_data = SigArray([ intbv(v)[data_w:] for v in range(2 ** addr_w)],
        name='ram_sig', init=True)

    inst = mem(clk=c, we=wren, addr_r=ra, addr_w=wa, din=a, dout=q, MODE=False, ram = ram_data,
        AWIDTH=addr_w, DWIDTH=data_w, TRANSPARENCY = TRANSPARENT)

    return inst

## Mapping to hardware

**Note** myIRL inference does *not yet* detect built-in transparency, (redundant) FFs will be inferred.

In [20]:
from myirl.targets import pyosys

In [21]:
DATA_WIDTH = 32

Elaborate memory unit and emit CXXRTL code:

In [22]:
def convert(dw, aw = 7):

    tgt = pyosys.RTLIL("memtest%d" % dw)

    MEM = r1w1

    tb = memtest(data_w=dw, mem = MEM, addr_w = aw, TRANSPARENT = True)
    d = tb.elab(tgt, elab_all = True)
    d = d[0]
    d.run("hierarchy -check")
    d.run("stat", capture = None)
    d.run("write_rtlil mem8.il")
    d.run("debug memory -nomap; debug opt")
    
    return d

In [23]:
d = convert(DATA_WIDTH, 7)

DEBUG IGNORING SET INIT FOR CLKSIGNAL


CALLED MEMORY INSTANCE 32
 Adding module with name `r1w1` 
DEBUG: ARRAY `ram_sig` TYPE <class 'myirl.lists.SigArray'> not instancing (yet)
 PARAM PRIORITY --> 48 
 PARAM WORDS --> 128 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 32 
 PARAM MEMID --> $mem_ram_sig 
 PARAM CLK_ENABLE --> False 
 PARAM CLK_POLARITY --> True 
 PARAM TRANSPARENT --> False 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 32 
 PARAM MEMID --> $mem_ram_sig 
 PARAM CLK_ENABLE --> True 
 PARAM CLK_POLARITY --> True 
 PARAM PRIORITY --> 48 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 32 
 PARAM MEMID --> $mem_ram_sig 
DEBUG TOP LEVEL WRITE MEM ram_sig_addr_w
 FINALIZE implementation `r1w1` of `r1w1` 
Dumping module `\r1w1_1'.

-- Running command `tee -q write_cxxrtl -namespace r1w1 -print-wire-types -header /tmp/myirl_r1w1_27qat2ex/r1w1_rtl.cpp' --

-- Running command `tee -q hierarchy -top \r1w1' --

-- Running command `tee -q hierarchy -check' --

-- Running command `stat' --

30. Printing statistics.

=== r1w1 ===

   Number of wires: 

     DEBUG SLICE READ MEMORY to dout AWIDTH:7 DWIDTH:32
DEBUG ADD MEM_INIT 7:32
     DEBUG SLICE WRITE MEMORY to ram_sig_addr_w AWIDTH:7 DWIDTH:32


 OPT_MUXTREE pass (detect dead branches in mux trees).
Running muxtree optimizer on module \r1w1..
  Creating internal representation of mux trees.
  No muxes found in this module.
Removed 0 multiplexer ports.

33.4. Executing OPT_REDUCE pass (consolidate $*mux and $reduce_* inputs).
  Optimizing cells in module \r1w1.
Performed a total of 0 changes.

33.5. Executing OPT_MERGE pass (detect identical cells).
Finding identical cells in module `\r1w1'.
Removed a total of 0 cells.

33.6. Executing OPT_DFF pass (perform DFF optimizations).

33.7. Executing OPT_CLEAN pass (remove unused cells and wires).
Finding unused cells or wires in module \r1w1..

33.8. Executing OPT_EXPR pass (perform const folding).
Optimizing module r1w1.

33.9. Finished OPT passes. (There is nothing left to do.)


## Testing hardware generation

Note: Post map simulation may require external memory models of blackbox Vendor Primitives.

In [24]:
TECHMAP = '/usr/share/yosys'

def synth_ecp5(d, libmap = True):
    # Read blackbox cells for awareness of DP16KD:
    d.run("read_verilog -lib -specify %s/ecp5/cells_sim.v %s/ecp5/cells_bb.v" % (TECHMAP, TECHMAP))

    if libmap: # New libmap procedure
        d.run("memory_libmap -lib %s/ecp5/brams.txt" % TECHMAP)        
    else:
        # First try DP16KD mapping:
        d.run("memory_bram -rules %s/ecp5/brams.txt" % TECHMAP)
    
    d.run("techmap -map %s/ecp5/brams_map.v" % TECHMAP)

    # Remaining (addr_w <= 8 bit) to LUT RAM:
    if libmap:
        d.run("memory_libmap -lib %s/ecp5/lutrams.txt" % TECHMAP)
    else:
        d.run("memory_bram -rules %s/ecp5/lutrams.txt" % TECHMAP)
        
    d.run("techmap -map %s/ecp5/lutrams_map.v" % TECHMAP)
    d.run("opt_clean")
    
def synth_gatemate(d, libmap = True):
    d.run("read_verilog -lib -specify %s/gatemate/cells_sim.v %s/gatemate/cells_bb.v" % (TECHMAP, TECHMAP))

    if libmap: # New libmap procedure
        d.run("memory_libmap -lib %s/gatemate/brams.txt" % TECHMAP)        
    else:
        d.run("memory_bram -rules %s/gatemate/brams.txt" % TECHMAP)
    d.run("techmap -map %s/gatemate/brams_map.v" % TECHMAP)

    d.run("opt")
    
d.run("ls", capture = None)

"\n-- Running command `ls' --\n\n1 modules:\n  r1w1\n"

In [25]:
synth_gatemate(d)
d.run("stat", capture = None)

d.run("write_rtlil mapped.il")

# d.display_rtl(selection = '*', fmt = 'dot')


-- Running command `ls' --

1 modules:
  r1w1

-- Running command `tee -q read_verilog -lib -specify /usr/share/yosys/gatemate/cells_sim.v /usr/share/yosys/gatemate/cells_bb.v' --

-- Running command `tee -q memory_libmap -lib /usr/share/yosys/gatemate/brams.txt' --

-- Running command `tee -q techmap -map /usr/share/yosys/gatemate/brams_map.v' --

-- Running command `tee -q opt' --

-- Running command `stat' --

39. Printing statistics.


"\n-- Running command `tee -q write_rtlil mapped.il' --\n"

In [26]:
#from yosys import display
#display.display_dot("memtest%d" % DATA_WIDTH)

In [27]:
d.write_verilog("test1")


=== r1w1 ===

   Number of wires:                 12
   Number of wire bits:            187
   Number of public wires:           6
   Number of public wire bits:      80
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  6
     $and                            1
     $dff                            2
     $eq                             1
     $mux                            1
     CC_BRAM_20K                     1


-- Running command `tee -q write_rtlil mapped.il' --

-- Running command `ls; check' --

41. Executing CHECK pass (checking for obvious problems).
Found and reported 0 problems.

-- Running command `hierarchy -check' --

42. Executing HIERARCHY pass (managing design hierarchy).

-- Running command `write_verilog test1.v' --

43. Executing Verilog backend.

43.1. Executing BMUXMAP pass.

43.2. Executing DEMUXMAP pass.


Define a few custom targets:

In [28]:
class CustomTarget(pyosys.RTLIL):
    def __init__(self, *args):
        super().__init__(*args)
        print("Selecting %s" % self.name)
        self.debug = True
        
    def finalize(self, top, objs = None):
        print("FINALIZE")
        tname = top.name
        design = self._design
        design.run("hierarchy -top %s" % tname)
        self.synth(design)
        design.run('stat', capture = None)
        # self.write_cxxrtl(top)
        return [ design ]

class GateMateTarget(CustomTarget):
    name = "GateMate"
    
    def synth(self, design):
        return synth_gatemate(design)

class ECP5Target(CustomTarget):
    name = "LatticeECP5"
    
    def synth(self, design):
        design.run("read_verilog ../library/tech/lattice/ecp5u/DP16KD.v")
        return synth_ecp5(design)        

Then run post-map simulation on one of them.

**Note**: For the ECP5 target, you need a *synthesizeable* variant of the `DP16KD.v` entity, or co-simulate using iverilog (currently not integrated into the pyrite simulator API). Let's try the GateMateTarget for now (which includes primitive whitebox models).

In [29]:
from cyrite import simulation
from yosys.simulator import CXXRTL

@simulation.sim.testbench(CXXRTL, time_unit = 'ns', target_class = GateMateTarget)
def tb_memtest(MODE = 0, STYLE = 1, data_w=16, addr_w=7, mem = r1w1):
    
    ClkSignal = simulation.ClkSignal
    Signal = simulation.Signal

    c = ClkSignal(name = 'clk')
    c.init = True
    
    M = (1 << (data_w - 1))

    wren = Signal(bool(), name = 'we')
    ra, wa = [ Signal(intbv()[addr_w:]) for n in ['addr_write', 'addr_read'] ]
    a, q = [ Signal(intbv()[data_w:]) for n in ['a', 'q'] ]

    ram_sig = SigArray([intbv(M | v)[data_w:] for v in range(2 ** addr_w)],
        name='ram_sig', init=True)

    # print(ram_sig[0].size())

    inst = mem(clk=c,
        we=wren,
        addr_r=ra, addr_w=wa, din=a, dout=q, MODE=False, ram = ram_sig,
        AWIDTH=addr_w, DWIDTH=data_w,
        TRANSPARENCY = True)

    @simulation.always(simulation.delay(2))
    def clkgen():
        c.next = ~c

    def write(addr, data):
        print("WRITE", addr, data)
        yield c.negedge
        wa.next = addr
        a.next = data
        wren.next = True
        yield c.negedge
        wren.next = False

    def write_verify(addr, data, value):
        print("WRITE VERIFY", addr)
        yield c.negedge
        wa.next = addr
        a.next = data
        wren.next = True
        yield c.negedge
        # print("DEBUG Q", bin(int(q)))
        assert int(q) == value
        wren.next = False

    @simulation.sequence
    def stim():
        ra.next = 0x20
        wa.next = 0x00

        # Write and verify that we're not bypassing:
        # i.e. the value is expected to be the initial one
        yield from write_verify(0x00, 0xaa, M | 0x20)
        ra.next = 0x00
        yield c.negedge
        # Now make sure we're getting the written value
        assert int(q) == 0xaa

        # ra == wa, verify transparency bypass:
        ra.next = 0x40
        yield from write_verify(0x40, 0x55, 0x55)
        yield c.negedge
        ra.next = 0x00
        yield c.negedge
        assert int(q) == 0xaa
        ra.next = 0x40
        yield c.negedge
        assert int(q) == 0x55
        ra.next = 0x1a
        yield c.negedge
        assert int(q) == M | 0x1a # Initial

        print("SIM DONE")

    return simulation.instances()


In [30]:
r1w1.ctx.components

['r1w1u_1u_1u_7u_7u_32u_32c_128_0_32_7_1']

In [31]:
t = tb_memtest(data_w = 10)

 Module r1w1: Existing instance r1w1, rename to r1w1_1 
CALLED MEMORY INSTANCE 10


DEBUG IGNORING SET INIT FOR CLKSIGNAL
DEBUG: not handling type <class 'myirl.emulation.myhdl.wrapped_wrapper'> in co-simulation. Your Cosimulation may not run correctly.
DEBUG: not handling type <class 'myirl.lists.SigArray'> in co-simulation. Your Cosimulation may not run correctly.


In [32]:
t.debug()
t.run(100)
# t.design.run("write_rtlil memtest.il")

Selecting GateMate
 Adding module with name `r1w1_1` 
DEBUG: ARRAY `ram_sig` TYPE <class 'myirl.lists.SigArray'> not instancing (yet)
 PARAM PRIORITY --> 48 
 PARAM WORDS --> 128 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 10 
 PARAM MEMID --> $mem_ram_sig 
 PARAM CLK_ENABLE --> False 
 PARAM CLK_POLARITY --> True 
 PARAM TRANSPARENT --> False 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 10 
 PARAM MEMID --> $mem_ram_sig 


     DEBUG SLICE READ MEMORY to dout AWIDTH:7 DWIDTH:10
DEBUG ADD MEM_INIT 7:10
     DEBUG SLICE WRITE MEMORY to ram_sig_addr_w AWIDTH:7 DWIDTH:10


 PARAM CLK_ENABLE --> True 
 PARAM CLK_POLARITY --> True 
 PARAM PRIORITY --> 48 
 PARAM ABITS --> 7 
 PARAM WIDTH --> 10 
 PARAM MEMID --> $mem_ram_sig 
DEBUG TOP LEVEL WRITE MEM ram_sig_addr_w
FINALIZE
Dumping module `\r1w1'.

-- Running command `tee -q hierarchy -top r1w1_1' --

-- Running command `tee -q read_verilog -lib -specify /usr/share/yosys/gatemate/cells_sim.v /usr/share/yosys/gatemate/cells_bb.v' --

-- Running command `tee -q memory_libmap -lib /usr/share/yosys/gatemate/brams.txt' --

-- Running command `tee -q techmap -map /usr/share/yosys/gatemate/brams_map.v' --

-- Running command `tee -q opt' --

-- Running command `stat' --

50. Printing statistics.

=== r1w1_1 ===

   Number of wires:                 10
   Number of wire bits:             58
   Number of public wires:           6
   Number of public wire bits:      36
   Number of memories:               1
   Number of memory bits:         1280
   Number of processes:              0
   Number of cells:             

Tolerate exception: ('Module with name `r1w1_1` already existing', 'While converting `[Instance r1w1_1 I/F: [// ID: r1w1_0 ]]`')
Using '/tmp/myirl_r1w1_lqmxoloy/' for output


running build_ext
building 'runtime.r1w1' extension
creating build/temp.linux-x86_64-3.10/tmp/myirl_r1w1_lqmxoloy
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=r1w1 -Iruntime -I/tmp/myirl_r1w1_lqmxoloy/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/myirl_r1w1_lqmxoloy/r1w1.cpp -o build/temp.linux-x86_64-3.10/tmp/myirl_r1w1_lqmxoloy/r1w1.o
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=r1w1 -Iruntime -I/tmp/myirl_r1w1_lqmxoloy/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/myirl_r1w1_lqmxoloy/r1w1_rtl.cpp -o build/temp.linux-x86_64-3.10/tmp/myirl_r1w1_lqmxoloy/r1w1_rtl.o
g++ -pthread -shared -Wl,--strip-all build/temp.linux-x86_64-3.10/tmp/myirl_r1w1_lqmxoloy/r1w1.o build/temp.linux-x86_64-3.10/tmp/myirl_r1w1_lqmxoloy/r1w1_rtl.o -L/usr/local/lib -o build/lib.linux-x86_64-3.10/runtime/r1w1.cp

DEBUG STOP PROCESS stim


1

# Duplex read, single write

This test is now part of the cyrite library [TDPRAM](tdpram.ipynb) 